# A3: A\*, IDS, and Effective Branching Factor

# Table of contents

2. [Overview](#overview)

2. [A Search](#a*)

2. [Iterative Deepening Search](#ids)

4. [Effective Branching Factor](#ebf)

5. [8-Puzzle](#8p)

 5.1 [printPath_8p](#printPath_8p)
 
 5.2 [findBlank_8p](#findBlank_8p)
 
 5.3 [actionsF_8p](#actionsF_8p)
 
 5.4 [takeActions_8p](#takeActionF_8p)
 
 5.5 [goalTestF_8p](#goalTestF_8p)
 
6. [Simple functions](#simple)

7. [Heuristic functions](#heuristic)

    7.1 [H1: Static value of zero](#h1_8p)
    
    7.2 [H2: Manhatthan distance](#h2_8p)
    
    7.3 [H3: Conflict lines](#h3_8p)
    
    7.4 [H4: Missplaced tiles](#h4_8p)
8. [Run experiment](#runExp)
 


<a id=overview></a>
## Overview

For this assignment, implement the Recursive Best-First Search
implementation of the A\* algorithm given in class.  Name this function `aStarSearch`.  Also in this notebook include your `iterativeDeepeningSearch` functions.  Define a new function named `ebf` that returns an estimate of the effective
branching factor for a search algorithm applied to a search problem.

So, the required functions are

   - `aStarSearch(startState, actionsF, takeActionF, goalTestF, hF)`
   - `iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth)`
   - `ebf(nNodes, depth)`, returns number of nodes expanded and depth reached during a search.

Apply `iterativeDeepeningSearch` and `aStarSearch` to several eight-tile sliding puzzle
problems. Compare their results by displayng
solution path depth, number of nodes 
generated, and the effective branching factor, and discuss the results.  Do this by defining the following function that prints the table as shown in the example below.

   - 'runExperiment(goalState1, goalState2, goalState3, [h1, h2, h3])

## Heuristic Functions

For `aStarSearch` use the following two heuristic functions, plus one more of your own design, for a total of three heuristic functions.

  * `h1(state, goal)`: $h(state, goal) = 0$, for all states $state$ and all goal states $goal$,
  * `h2(state, goal)`: $h(state, goal) = m$, where $m$ is the Manhattan distance that the blank is from its goal position,
  * `h3(state, goal)`: $h(state, goal) = ?$, that you define.  It must be admissible, and not constant for all states.

## Comparison

Apply all four algorithms (`iterativeDeepeningSearch` plus `aStarSearch` with the three heuristic
functions) to three eight-tile puzzle problems with start state

$$
\begin{array}{ccc}
1 & 2 & 3\\
4 & 0 & 5\\
6 & 7 & 8
\end{array}
$$

and these three goal states.

$$
\begin{array}{ccccccccccc}
1 & 2 & 3  & ~~~~ & 1 & 2 & 3  &  ~~~~ & 1 & 0 &  3\\
4 & 0 & 5  & & 4 & 5 & 8  & & 4 & 5 & 8\\
6 & 7 & 8 &  & 6 & 0 & 7  & & 2 & 6 & 7
\end{array}
$$

Print a well-formatted table like the following.  Try to match this
format. 

           [1, 2, 3, 4, 0, 5, 6, 7, 8]    [1, 2, 3, 4, 5, 8, 6, 0, 7]    [1, 0, 3, 4, 5, 8, 2, 6, 7] 
    Algorithm    Depth  Nodes  EBF              Depth  Nodes  EBF              Depth  Nodes  EBF          
         IDS       0      0  0.000                3     43  3.086               11 225850  2.954         
        A*h1       0      0  0.000                3    116  4.488               11 643246  3.263         
        A*h2       0      0  0.000                3     51  3.297               11 100046  2.733         

Of course you will have one more line for `h3`.

<a id=a*></a>
# A* Search
The type of search that is done by this algorithm is an informed search as it chooses the next node to visit based on the domain knowledge given by the value of an function related to each step of the problem. 

The crucial component  in the A* search consist in the value of each state or node. This allows to reduce the time invested in reaching the goal state exponentially, given a certain start state. To compute the value for node "state", the following function is used:
   <br>
   <br> f(state) = g(state) + h(state)
   
   <br> having g as the cost to get to the goal state from the given start state, and h is the heuristic function, which will calculate the cost of all paths from the given state.
   
The  A* search function for this assigment was developed using the recursive implementation of the algorithm  provided by Dr. Chuck Anderson. The code was modified to count the number of nodes expanded in each search and the depth from the initial state to the goal state. Each state is represented as an instance of the class "Node" that contains the specific state and its f,g and h values.

The heurisctic functions used to perform this type of search will be described later.
 


In [1]:
class Node:
    def __init__(self, state,f=0, g=0 ,h=0):
        self.state = state
        self.f = f
        self.g = g
        self.h = h  
        self.parent=Node

    def __repr__(self):
        return "Node(" + repr(self.state) + ", f=" + repr(self.f) + \
               ", g=" + repr(self.g) + ", h=" + repr(self.h) + ")"

def aStarSearch(startState, actionsF, takeActionF, goalTestF, hF):
    global nNodes
    global depth
    nNodes=0
    depth=0
    h = hF(startState)
    startNode = Node(state=startState, f=0+h, g=0, h=h)
    a= aStarSearchHelper(startNode, actionsF, takeActionF, goalTestF, hF, float('inf'))
    depth = len(a[0])-1
    
    return a

def aStarSearchHelper(parentNode, actionsF, takeActionF, goalTestF, hF, fmax):
    global nNodes
    if goalTestF(parentNode.state):
        return ([parentNode.state], parentNode.g)
    ## Construct list of children nodes with f, g, and h values
    actions = actionsF(parentNode.state)
    if not actions:
        return ("failure", float('inf'))
    children = []
    for action in actions:
        (childState,stepCost) = takeActionF(parentNode.state, action)
        h = hF(childState)
        g = parentNode.g + stepCost
        f = max(h+g, parentNode.f)
        childNode = Node(state=childState, f=f, g=g, h=h)
        children.append(childNode)
        nNodes+=1

    while True:
        # find best child
        children.sort(key = lambda n: n.f) # sort by f value
        bestChild = children[0]
        if bestChild.f > fmax:
            return ("failure",bestChild.f)
        # next lowest f value
        alternativef = children[1].f if len(children) > 1 else float('inf')
        # expand best child, reassign its f value to be returned value
        result,bestChild.f = aStarSearchHelper(bestChild, actionsF, takeActionF, goalTestF,
                                            hF, min(fmax,alternativef))
        if result is not "failure":               
            result.insert(0,parentNode.state)      
            return (result, bestChild.f) 
        

<a id=ids></a>
# Iterative Deepening Search

The *iterativeDeepeningSearch* algorithm will generate a path between a start and a goal state if it exists.
This algorithm will call the *depthLimitedSearch* algorithm as many times as the value *maxDepth* shows, using a loop. If in one run within the loop a path is returned by the *depthLimitedSearch* algorithm, then the execution will not continue because it already found a solution. If in one of the call within the loop a  *cutoff* is returned, it means that with the given depth limit, the goal state was not found, so the loop will continue with a higher depth limit to try to reach the goal state. If the goal state is not in the graph, or it is not reachable from the given start state, then the algorithm will return  *failure*.


In [46]:
def iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth):
    global nNodes
    global depth
    nodeCounter=0
    nNodes=0
    depth=0
    path= 'cutoff'
    p=[]
    resultFound='false'
    for x in range(maxDepth+1):
        p= depthLimitedSearch(startState, goalState, actionsF, takeActionF, x)
        nodeCounter+=nNodes
        if p!='cutoff':
            depth=0
            if startState!=goalState:
                depth=len(p)-1
                nNodes=nodeCounter
            return p
        
    if p=='cutoff':
        return 'cutoff'
    if p=='failure':
        return 'failure'

The *depthLimitedSearch* algorithm will receive a depth limit value that indicates how deep it should go in that run. To know how deeper is the node been consulted in that moment, it will check the *expanded* dictionary and will calculate how many steps are there from that node to the root. If it finds that the depth limit has been reached, it will not insert that node's children to the unexpanded list and will continue checking the neighbors.
To generate the children for a certain node, the algorithm will use the functions *actionsF* and *takeActionF*.
The purpose of the first function is to generate all possible actions that can be applied to a given state and the cost of that action. The second function will apply each of those actions to the state in order to generate all the children for that specific node.Every time a new state is generated from this actions, a global counter named *nNodes* will be incremented to keep track of how many nodes were expanded in a specific search.



In [45]:
def depthLimitedSearch(startState , goalState, actionsF, takeActionF,depthLimit):
    global nNodes
    expanded={}
    unexpanded= [(startState,"None")]
    solutionPath=[]
    existingChildren=[]
    depth=0
    needTuple='false'
    end='true'
    #If startState is the goalState, return the list containing just startState
    if startState==goalState:
        return startState
    else:
        while unexpanded:
            #print unexpanded
            #print expanded
            
            childrenAll=[]

            #Pop from the end of unExpanded a (state, parent) pair.
            (state,parent)= unexpanded.pop()
            if(state not in existingChildren):
                existingChildren.append(state);
            if(parent not in existingChildren):
                existingChildren.append(parent);

            #Add the node to the expanded dictionary, indexed by its state.
            if isinstance(state, list):
                state= tuple(state)
                needTuple= 'true'     
                
            expanded[state]=parent
            
            #check if depth limit has been reached
            searchParent=parent
            predecesor='true'
            steps=0
            while predecesor=='true':
                if searchParent in expanded.keys():
                    steps=steps+1
                    searchParent=expanded[searchParent]              
                else:
                    predecesor='false'
            children=[]
            
            
            if steps < depthLimit:
                #generates posible actions
                listActions=actionsF(state)
                #Generates successor depending on the posible moves  
                for eachAction in listActions:
                    c,cost=takeActionF(state, eachAction)
                    nNodes+=1
                    #c=takeActionF(state,eachAction)
                    if c not in existingChildren:
                        children.append(c)  
                        
                #reverse children list 
                children=list(reversed(children))
                #insert children at the end of unexpanded list
                for p in children:
                    #state=list(state)
                    unexpanded.append((p,state))
            else:
                 #generates posible actions
                listActions=actionsF(state)
                #Generates successor depending on the posible moves  
                for eachAction in listActions:
                    c,cost=takeActionF(state, eachAction)
                    if c not in existingChildren:
                        children.append(c) 
                if len(children) > 0:
                    end='false'
                children=[]     
                
        

     
            #If the goal has been found (in python, goalState is in children):
            if goalState in children:  
                solutionPath.append(goalState)
                solutionPath.append(state)
                while parent in expanded.values():
                    if parent=="None":
                        break
                    solutionPath.append(parent)
                    parent=expanded[parent]
                if  needTuple=='true':
                    newSolutionPath=[]
                    for x in solutionPath:
                        lst= list(x)
                        newSolutionPath.append(lst)               
                    solutionPath= newSolutionPath 
                return list(reversed(solutionPath))
            
         
            
        if  end=='true':
            return "failure"
        else:
            return "cutoff"

<a id=ebf></a>
# Effective Branching Factor

The effective branching factor function  (ebf) is used to evaluate the effectiveness of a certain search using different goal states and heuristic functions in the case of A* search and different goal states and depth limits in the case of the iterative deepening search.

After every execution of the search algorithm, a certain value has been assigned to the global variables *nNodes* and *depth*. The first value represents the total number of nodes processed during the search and the second  represents the depth in which the goal state was found.

The purpose of this function is to evaluate how accurate is the heuristic function used to solve a A* search in relation to the real cost of going from the start state to the goal state.


In [4]:
def ebf(nodes, depth, precision=0.01):
    result=0
    d=depth
    b=0
    while (nodes-result) > precision:
        result=(1-b**(d+1))/(1-b)
        b+=precision
        
    return b


In [5]:
ebf(100, 12, 0.01)

1.320000000000001

In [6]:
ebf(0, 0)

0

In [7]:
ebf(1, 1,)

0.01

<a id=8p></a>
# 8-Puzzle
The following are some of the functions used by the A* search algorithm and the iterative deepening search algorithm to solve the 8 puzzle


### printPath_8p
<a id=printPath_8p></a>
This function is used to print a certain state of the 8-puzzle or n-puzzle in a more readeble way.


In [8]:
def printPath_8p(startState):
    length= len(startState)
    dim=0
    for c in range(length):
        counter=c**2
        if counter==length: 
            dim=c              
    for i in range(dim):
        for j in range(dim):
             print(startState[i*dim+j]), 
        print ('')

startState=[1, 0, 3, 4, 2, 5, 6, 7, 8]
printPath_8p(startState)

1 0 3 
4 2 5 
6 7 8 


<a id=findBlank_8p></a>
###     findBlank_8p
This function will return the position (row and column) in which the blank tile is located.


In [9]:
def findBlank_8p(startState):
    blankPosition=[]
    length= len(startState)
    dim= 0
    for c in range(length):
        counter=c**2
        if counter==length: 
            dim=c 
    if 0 in startState:
       
        for row in range(dim):          
             for column in range(dim): 
                    if startState[row*dim+column]==0:
                        blankPosition.append(row)
                        blankPosition.append(column)
                        
    return blankPosition

findBlank_8p([1, 2, 3, 4, 0, 5, 6, 8, 7])

[1, 1]

<a id=actionsF_8p></a>
###      actionsF_8p

The function *actionsF_8p* is used to generate all the possible directions where the zero can move in a certain state of the puzzle. The options will be returned in the the following order: left, right, up, down if all of them are valid moves.

The way the algorithm works is: first assumes that all moves are valid, then it will check the borders and will remove any of the movements that is not valid for that state.

For each valid action, the algorithm will return a pair containing the action an a number 1, indication that the cost to get from the given state to the child state using the corresponding action will be 1, or that the number of moves needed is 1.


In [10]:
def actionsF_8p(startState):
   
    action=[]
    
    #gets puzzle demension
    dim=0
    length= len(startState)
    for c in range(length):
        counter=c**2
        if counter==length: 
            dim=c  
    
    blankPosition=findBlank_8p(startState)
    row=blankPosition[0]
    column=blankPosition[1]
    blankPosition=row*dim+column
   
    #check if there is blank position
    if blankPosition !=-1:
        action=['left', 'right', 'up', 'down']
      

        #check left border
        for row in range(dim):
                if blankPosition in [row*dim]:
                        action.remove('left')
                        
        #check right border
        for row in range(dim):
            if blankPosition in [(row*dim)+(dim-1)]:
                     action.remove('right')
        
        #check top border
        if blankPosition -dim <0:
            action.remove('up')
            
        #check bottom border
        if blankPosition +dim >=len(startState):
            action.remove("down")        
    
    actionCost=[]
    for ac in action:
        actionCost.append((ac,1))
    return actionCost

actionsF_8p([1, 2, 3, 4, 0, 5, 6, 8, 7])

[('left', 1), ('right', 1), ('up', 1), ('down', 1)]

<a id=takeActionF_8p></a>
###     takeActionF_8p
The *takeActionF_8p* function will receive a state and an action, consisting of a pair of the direction and the cost, and will generate a new state applying that action to the currect state.


In [11]:
def takeActionF_8p(startState, act):

    newState=[]
    invalidLeft='false'
    invalidRight='false'
    invalidDown='false'
    invalidUp='false'
    #gets puzzle demension
    dim=0
    length= len(startState)
    for c in range(length):
        counter=c**2
        if counter==length: 
            dim=c  
    
    blankPosition=findBlank_8p(startState)
    row=blankPosition[0]
    column=blankPosition[1]
    blankPosition=row*dim+column
    
    
    #check left border
    for r in range(dim):
        if blankPosition in [r*dim]:
            invalidLeft='true'
                   

    #check right border
    for row in range(dim):
        if blankPosition in [(r*dim)+(dim-1)]:
            invalidRight='true'

    #check top border
    if blankPosition -dim <0:
         invalidUp='true'

    #check bottom border
    if blankPosition +dim >=len(startState):
        invalidDown='true'  
    
    action=act[0]
    pos=0
    for x in startState:
        newState.append(x)
    if action=='left' and invalidLeft=='false':
        pos=blankPosition
        pos=pos-1
        newState[blankPosition]=startState[pos]
        newState[pos]=0
    if action=='right'and invalidRight=='false':
        pos=blankPosition
        pos=pos+1
        newState[blankPosition]=startState[pos]
        newState[pos]=0
    if action=='up' and invalidUp=='false':
        pos=blankPosition
        pos=pos-dim
        newState[blankPosition]=startState[pos]
        newState[pos]=0
    if action=='down' and invalidDown=='false':
        pos=blankPosition
        pos=pos+dim      
        newState[blankPosition]=startState[pos]
        newState[pos]=0
      
    return (newState,act[1])
takeActionF_8p([1, 2, 3, 4, 0, 5, 6, 8, 7],('left', 1) )

([1, 2, 3, 0, 4, 5, 6, 8, 7], 1)

<a id=goalTestF_8p></a>
###      goalTestF_8p
This function is used to evaluate if a certain state is the goal state.

In [12]:

def goalTestF_8p(s,goal):
    return s == goal


<a id=simple></a>
### Simple functions
The following functions are used to solve  A* and IDS searches with values different than an 8-puzzle, for example graphs with characters or numbers.

In [13]:

succs= {'a': ['b', 'c'], 
              'b':['e', 'f', 'g'], 
              'b':['a'], 
              'c':['h'],
              'h':['i'], 
              'i':['j', 'k', 'l'],
              'k':['z']}
                  





In [14]:
def actionsF_simple(s):  
    try:                                      
        ## step cost of each action is 1
        return [(succ,1) for succ in succs[s]]
    except KeyError:
        return []
actionsF_simple('a')


[('b', 1), ('c', 1)]

In [15]:
def takeActionF_simple(state, action):
    return action
takeActionF_simple('a', ('left',1))

('left', 1)

In [16]:
def goalTestF(s):
    return s == goal

In [17]:
def h1(s):
    return 0

In [18]:
def h_simple(s,g):
    return 0


In [19]:
def goalTestF_simple(s,goal):
    return s == goal

<a id=heuristic></a>
## Heuristic functions
Heuristic functions are used in informed search problems. The objective of this functions is to guide the search to a certain path in which the cost to get to the goal state is the lesser possible for a certain state giving an estimate of the sum of the remaining step costs to a goal.

<a id=h1_8p></a>
### H1: Static value of zero
This is a static heuristic function because it will always return a value of 0 no matter what the start and goal states are.

In [20]:
def h1_8p(state, goal):
    return 0

<a id=h2_8p></a>

### H2: Manhatthan distance

A commun implementation of this algorithm describes that for a certain tile in a n-puzzle the Manhattan Distance will be the distance between its state and its position in the goal state.It means that, for a certain state the Manhattan distance will be the sum of the Manhattan distances of all the tiles without considering the blank tile.
However, this implementation of the Manhattan distance heuristic function will consider only the number of spaces that the blank space is away from its goal state. Contrary to the first description, where every misplaces tile is considered to calculate the value returned by the heuristic function.





In [21]:
def h2_8p(startState, goalState):
    manhattanDistance=0
    
    statePosRow=0
    statePosCol=0
    goalPosRow=0
    goalPosCol=0
    
    stateItem=0
    goalItem=0
    
    length= len(startState)
    dim= 0
    for c in range(length):
        counter=c**2
        if counter==length: 
            dim=c 
    for row in range(dim):          
        for column in range(dim):
            stateItem=startState[row*dim+column]
            goalItem=goalState[row*dim+column]
            if stateItem!=goalItem and stateItem==0:
                statePosRow=row
                statePosCol=column
                goalPosRow,goalPosCol=findPosition(goalState,stateItem)
                x=abs(statePosRow-goalPosRow)+abs(statePosCol-goalPosCol)
                manhattanDistance+=x
            
    return manhattanDistance

#h2_8p([7,2,4,5,0,6,8,3,1], [0,1, 2, 3, 4, 5, 6, 7, 8] )
            

In [22]:
#given an item and  a state, returns the position of that item in the puzzle
def findPosition(startState, item):
    position=[]
    length= len(startState)
    dim= 0
    for c in range(length):
        counter=c**2
        if counter==length: 
            dim=c 
    if item in startState:
       
        for row in range(dim):          
             for column in range(dim): 
                    if startState[row*dim+column]==item:
                        position.append(row)
                        position.append(column)
                        
    return position

#findPosition([1, 2, 4, 4, 5, 6, 7, 0, 8], 3)


<a id=h3_8p></a>
### H3: Linear conflict  
This heuristic function is used as a complement of the Manhathan distance heuristic function.

To determine if two tiles are having a linear conflict, the algorithm will check if both of them are in the same line, in this case, a line is represented by a row or column in the 8 puzzle. If both tiles are in the same line,  their goal states are also both in the same line and one of the tiles is blocked by the other tile, a linear conflict is present in that specific state of the puzzle.  

The main approach of this function is that considers movements that are not counted by the Manhattan Distance heuristic function. If there are two tiles having a linear conflict, two movements are required to put both tiles in their goal position, moving one of them down and up again. This steps are not present in the result of a estimated remaining cost to the goal state using the simple Manhattan distance function.

It is important to mention that the Manhattan distance function been used by the Linear conflict function, is the one that considers all the tiles that are not in place, excluding the blank tile. 

This algorithm does not add overestimation to the heuristic calculation of the remaining cost to get to the goal state, because it will not contemplate the same tile as part of more than one linear conflict, in this way the algorithm is just adding the two movements that the Manhattan distance dismisses for each misplaced tile, which retains the admissibility of the function.

Even if the version of Manhattan distance considering all the tiles but the blank is used, it never overestimates the true cost to the solution, because it it considering every misplaced tile, and each of those tiles must be moved one or more positions to get to their goal state.


In [23]:
def h3_8p(startState, goalState):
    result=0
    
    length= len(startState)
    dim= 0
    for c in range(length):
        counter=c**2
        if counter==length: 
            dim=c 
    #Row Conflicts
    for i in range(dim-1):
        result += findConflicts(startState, i, 1,goalState);

    #Column Conflicts
    for i in range(dim-1):
        result += findConflicts(startState, i, 0,goalState);

    return h5_8p(startState,goalState)+2*result;



In [24]:
def inConflict(index, a, b, indexA, indexB,  dimension,goalState):
    indexGoalA = -1;
    indexGoalB = -1;
    r=0
    
    length= len(goalState)
    dim= 0
    for c in range(length):
        counter=c**2
        if counter==length: 
            dim=c 
    
    for c in range(dim-1):
        if dimension == 1 and goalState[index*dim+c]==a:
            indexGoalA = c
        elif dimension == 1 and goalState[index*dim+c]==b:
            indexGoalB = c
            
        if dimension == 0 and goalState[c*dim+index]==a:
            indexGoalA = c
        elif dimension == 0 and goalState[c*dim+index]==b:
            indexGoalB = c;
                

    if ((indexGoalA >= 0 and indexGoalB >= 0) and 
    ((indexA < indexB and indexGoalA > indexGoalB) or
     (indexA > indexB and indexGoalA < indexGoalB))):
        r=2
    return r
#InConflict(0, 2, 3, 0, 1,  1, [1, 2, 3, 4, 5, 6, 7, 8, 0])

                        

In [25]:
def findConflicts(startState, i,  dimension,goalState):
    result = 0;
    tilesRelated = []
    
    length= len(startState)
    dim= 0
    for c in range(length):
        counter=c**2
        if counter==length: 
            dim=c 
    

    #Loop foreach pair of elements in the row/column
    h=0
    while h< dim-1 and h not in tilesRelated:
        k=h+1
        while k < dim and h not in tilesRelated:
            if dimension == 1 and startState[i*dim+h]==0:
                k+=1
                continue
            if dimension == 0 and startState[h*dim+i]==0:
                k+=1
                continue
            if dimension == 1 and startState[i*dim+k]==0:
                k+=1
                continue
            if dimension == 0 and startState[k*dim+i]==0:
                k+=1
                continue
                
            if dimension ==1:
                moves =inConflict(i, startState[i*dim+h], startState[i*dim+k], h, k, dimension, goalState)
            else:
                moves=inConflict(i, startState[h*dim+i], startState[k*dim+i], h, k, dimension,goalState)
            
            if (moves == 0):
                k+=1
                continue
            result += 2;
            tilesRelated.append(h)
            tilesRelated.append(k)
            k+=1   
            break;
        h+=1
    return result
    
    
#findConflicts([2,3,1,4,5,6,0,8,7], 2,  1, [1,2,3,4,5,6,7,8,0])

### Manhattan Distance considering all the misplaced tiles
This implementation of Manhattan Distance heuristic will consider all the misplaced tiles in relation with the goal state, but without considering the blank space.

This is an admissible heuristic because even if it considers all the misplaced tiles, it will never overestimate the real cost because all the tiles must be moved at least one time form that specific state.


In [26]:
def h5_8p(startState, goalState):
    manhattanDistance=0
    
    statePosRow=0
    statePosCol=0
    goalPosRow=0
    goalPosCol=0
    
    stateItem=0
    goalItem=0
    
    length= len(startState)
    dim= 0
    for c in range(length):
        counter=c**2
        if counter==length: 
            dim=c 
    for row in range(dim):          
        for column in range(dim):
            stateItem=startState[row*dim+column]
            goalItem=goalState[row*dim+column]
            #consideres all the tiles but the blank 
            if stateItem!=goalItem and stateItem!=0:
                statePosRow=row
                statePosCol=column
                goalPosRow,goalPosCol=findPosition(goalState,stateItem)
                x=abs(statePosRow-goalPosRow)+abs(statePosCol-goalPosCol)
                manhattanDistance+=x
            
    return manhattanDistance

Here is an example of a result executing the Manhattan Distance considering all the misplaced tiles: 

In [27]:
h5_8p([3, 2, 1, 4, 0, 5, 6, 7, 8],[1, 2, 3, 4, 0, 5, 6, 7, 8])

4

After doing this, the linear conflict function can be executed with the same start and goal states to show that 4 units were added to the result.  The result of linear conflict heuristic function will be:

<br> Manhattan Distance considering all the misplaced tiles + 2 * linear conflicts 

In [28]:
h3_8p([3, 2, 1, 4, 0, 5, 6, 7, 8],[1, 2, 3, 4, 0, 5, 6, 7, 8])

8

<a id=h4_8p></a>

### H4: Misplaced tiles
This heuristic function will return a value as a result of the sum of all the misplaced tiles that are found in a certain state of the puzzle.
It is an admissible heuristic because for each of the misplaced tiles will add just 1 to the result, and the real cost to the goal state will include move each of those misplaced tiles at least one time, thus, it is not overstimating the real cost.


In [29]:
def h4_8p(startState,goalState):
    misplacedTiles=0
 
    length= len(startState)
    dim= 0
    for c in range(length):
        counter=c**2
        if counter==length: 
            dim=c 
    for row in range(dim):          
        for column in range(dim):
            if startState[row*dim+column]!=goalState[row*dim+column]:
                misplacedTiles+=1

            
    return misplacedTiles

#h3([7,2,4,5,0,6,8,3,1], [3,2,4,5,6,0,8,7,1])'''
            

<a id=runExp></a>
## Run Experiment
In this section, some experiments are executed to compare the result of A* and IDS search algorithms with different goal states and different heuristic function. This will enable to possibility of compare the results to determine the performance of the heuristic functions.


In [49]:
def runExperiment(goalState1, goalState2, goalState3, heuristicFunctions):
    import pandas
    import numpy as np
    r=[]
    #startState=[3, 5, 1, 4, 0, 2, 6, 7, 8]
    startState=[1, 2, 3, 4, 0, 5, 6, 7, 8]
    
    
    iterativeDeepeningSearch(startState, goalState1, actionsF_8p, takeActionF_8p, 20)
    depth1=depth
    nNodes1=nNodes
    ebf1=ebf(nNodes1,depth1)

    
    iterativeDeepeningSearch(startState, goalState2, actionsF_8p, takeActionF_8p, 20)
    depth2=depth
    nNodes2=nNodes
    ebf2=ebf(nNodes2,depth2)
  

    
    iterativeDeepeningSearch(startState, goalState3, actionsF_8p, takeActionF_8p,20 )
    depth3=depth
    nNodes3=nNodes
    ebf3=ebf(nNodes3,depth3)
 


  
    r.append(([depth1,nNodes1,ebf1],[depth2,nNodes2,ebf2],[depth3,nNodes3,ebf3]))     

    for hF in heuristicFunctions:
           
            
            depth0=0
            nNodes0=0
            ebf0=ebf(nNodes0,depth0)
            aStarSearch(startState,actionsF_8p, takeActionF_8p,
            lambda s: goalTestF_8p(s, goalState1),
            lambda s: hF(s,goalState1))      
            depth1=depth
            nNodes1=nNodes
            ebf1=ebf(nNodes1,depth1)
            
            
            
          
            aStarSearch(startState,actionsF_8p, takeActionF_8p,
            lambda s: goalTestF_8p(s, goalState2),
            lambda s: hF(s,goalState2))   
            depth2=depth
            nNodes2=nNodes
            ebf2=ebf(nNodes2,depth2)
            
            
            aStarSearch(startState,actionsF_8p, takeActionF_8p,
            lambda s: goalTestF_8p(s, goalState3),
            lambda s: hF(s,goalState3))  
            depth3=depth
            nNodes3=nNodes
            ebf3=ebf(nNodes3,depth3)
            
            
            df=pandas
            pd=pandas
            
            r.append(([depth1,nNodes1,ebf1],[depth2,nNodes2,ebf2],[depth3,nNodes3,ebf3]))
 
    r0=r[0]
    r1=r[1]
    r2=r[2]
    r3=r[3]
    

    return pd.DataFrame([( goalState1,'', '','',goalState2 ,'', '','',goalState3,  '', '',''),
                        ( ' ','DEPTH', 'NODES','EBF','  ' ,'DEPTH', 'NODES','EBF',' ',  'DEPTH', 'NODES','EBF'),
                         ( ' ',r0[0][0], r0[0][1],r0[0][2],' ' ,r0[1][0], r0[1][1],r0[1][2],' ',  r0[2][0], r0[2][1],r0[2][2]),
                        ( ' ',r1[0][0], r1[0][1],r1[0][2],' ' ,r1[1][0], r1[1][1],r1[1][2],' ',  r1[2][0], r1[2][1],r1[2][2]),
                        ( ' ',r2[0][0], r2[0][1],r2[0][2],' ' ,r2[1][0], r2[1][1],r2[1][2],' ',  r2[2][0], r2[2][1],r2[2][2]),
                        ( ' ',r3[0][0], r3[0][1],r3[0][2],' ' ,r3[1][0], r3[1][1],r3[1][2],' ',  r3[2][0], r3[2][1],r3[2][2])], 
                       columns=['','','','','','','','','','','',''], index=['Goal State','Algorithms','IDS',heuristicFunctions[0].__name__,heuristicFunctions[1].__name__, heuristicFunctions[2].__name__])



## Comparing  Manhathan Distance with misplace tiles
#### h2:manhattan distance considering only the blank tile
#### h4:misplaced tiles
#### startState=[1, 2, 3, 4, 0, 5, 6, 7, 8]

In [50]:
runExperiment([1, 2, 3, 4, 0, 5, 6, 7, 8]   , [1, 2, 3, 4, 5, 8, 6, 0, 7]   , [1, 0, 3, 4, 5, 8, 2, 6, 7]  , [h1_8p, h2_8p, h4_8p])

,,,,,,,,,,,,
Goal State,"[1, 2, 3, 4, 0, 5, 6, 7, 8]",,,,"[1, 2, 3, 4, 5, 8, 6, 0, 7]",,,,"[1, 0, 3, 4, 5, 8, 2, 6, 7]",,,
Algorithms,,DEPTH,NODES,EBF,,DEPTH,NODES,EBF,,DEPTH,NODES,EBF
IDS,,0,0,0,,3,62,3.57,,11,48288,2.56
h1_8p,,0,0,0,,3,116,4.5,,11,643246,3.28
h2_8p,,0,0,0,,3,51,3.31,,11,100046,2.75
h4_8p,,0,0,0,,3,9,1.59,,11,5232,2.06


In this case the most effective heuristic function is the h3 because for all the cases presents the lowest effective branching factor. This result is expected because the h2 function is using the Manhattan distance heuristic in a really optimistic way, considering only the blank tile, while the h3 function considers all the misplaced tiles but without overestimating the real cost.

The h1 function will always have a greater EBF because it will no make any difference between the nodes to chose, since all of them in the same level will have the same *f* value, so the the search will choose the next node to visit randomly and not based on the posible lower cost to get to the goal state.

The  EBF for the IDS is lower than the EBF using the manhattan distance considering only the blank tile, because this heuristic will generate more nodes than the other version of the manhattan distance that consideres all the  tiles. Compared to the misplace tiles heuristic function, is worst because will generate more innecesary nodes and this heuristic function avoids generating this unnecesary nodes estimating a higher cost to the goal state but never overestimating it. 

The IDS will be better than the h1 heuristic function since this heuristic will generate a huge amount of unnecesary nodes because it has no guide to determine the best path, while the IDS is limited by the depth limit used in each of the executions. The depth limit avoids generating extra nodes limiting the search instead of covering a hole branch of a certain node considering that the goal state might not be in the branch. 


## Comparing both implementations of Manhattan Distance 
#### h2:manhattan distance considering only the blank tile
#### h5:manhattan distance considering all the misplaced tiles
#### startState=[1, 2, 3, 4, 0, 5, 6, 7, 8]

In [32]:
runExperiment([1, 2, 3, 4, 0, 5, 6, 7, 8]   , [1, 2, 3, 4, 5, 8, 6, 0, 7]   , [1, 0, 3, 4, 5, 8, 2, 6, 7]  , [h1_8p, h2_8p, h5_8p])

,,,,,,,,,,,,
Goal State,"[1, 2, 3, 4, 0, 5, 6, 7, 8]",,,,"[1, 2, 3, 4, 5, 8, 6, 0, 7]",,,,"[1, 0, 3, 4, 5, 8, 2, 6, 7]",,,
Algorithms,,DEPTH,NODES,EBF,,DEPTH,NODES,EBF,,DEPTH,NODES,EBF
IDS,,0,0,0,,3,28,2.62,,11,8419,2.16
h1_8p,,0,0,0,,3,116,4.5,,11,643246,3.28
h2_8p,,0,0,0,,3,51,3.31,,11,100046,2.75
h5_8p,,0,0,0,,3,9,1.59,,11,1172,1.78


In this experiment the difference between h2 and h5 is considerable, because h2 is only considering the blank tile, while h5 is less optimistic and considers all the misplace tiles and remians addmissible. Both of them are admissible, so the h5 function in this case seems to be more effective.

In this case as well, the IDS will be a bad option compared to the heuristic manhattan distance considering all the misplaced tiles.

## Manhattan Distance + Linear conflict 
#### h3:manhattan distance considering all the misplaced tiles+ linear conflict
#### h5:manhattan distance considering all the misplaced tiles
#### startState=1, 2, 3, 4, 0, 5, 6, 7, 8]

In [33]:
runExperiment([1, 2, 3, 4, 0, 5, 6, 7, 8]   , [1, 2, 3, 4, 5, 8, 6, 0, 7]   , [1, 0, 3, 4, 5, 8, 2, 6, 7]  , [h1_8p, h3_8p, h5_8p])

,,,,,,,,,,,,
Goal State,"[1, 2, 3, 4, 0, 5, 6, 7, 8]",,,,"[1, 2, 3, 4, 5, 8, 6, 0, 7]",,,,"[1, 0, 3, 4, 5, 8, 2, 6, 7]",,,
Algorithms,,DEPTH,NODES,EBF,,DEPTH,NODES,EBF,,DEPTH,NODES,EBF
IDS,,0,0,0,,3,28,2.62,,11,8419,2.16
h1_8p,,0,0,0,,3,116,4.5,,11,643246,3.28
h3_8p,,0,0,0,,3,9,1.59,,11,1172,1.78
h5_8p,,0,0,0,,3,9,1.59,,11,1172,1.78


Unfortunately an example with a start state containing a linear conflict was not possible to execute because if a linear conflict is added on purpose to the start state, the algorithm will take more than 20 minutes to complete the search, and no solution was found to solve this issue. My initial guess is that simply with that given start state the algorithm will take to long to find the goal state.

The result should be a ebf lower for the linear conflict heuristic because considering the 2 moves that the manhattan distances do not include, will make the function execute generating less unnecessary nodes but without overestimating the cost to find the goal state.


## Other examples

In [34]:
iterativeDeepeningSearch('a', 'z', actionsF_simple, takeActionF_simple, 10)

['a', 'c', 'h', 'i', 'k', 'z']

In [35]:
aStarSearch('a',actionsF_simple, takeActionF_simple,
            lambda s: goalTestF_simple(s, 'z'),
            lambda s: h_simple(s, 'z'))

(['a', 'c', 'h', 'i', 'k', 'z'], 5)

In [36]:
aStarSearch([1, 2, 3, 4, 5, 6, 7, 0, 8],
                     actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]),
                     lambda s: h1_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]))

([[1, 2, 3, 4, 5, 6, 7, 0, 8],
  [1, 2, 3, 4, 0, 6, 7, 5, 8],
  [1, 2, 3, 0, 4, 6, 7, 5, 8],
  [0, 2, 3, 1, 4, 6, 7, 5, 8]],
 3)

In [37]:
iterativeDeepeningSearch([5, 2, 8, 0, 1, 4, 3, 7, 6], 
                                 [0, 2, 3, 1, 4,  6, 7, 5, 8],
                                 actionsF_8p, takeActionF_8p, 10)

'cutoff'

In [38]:
%run -i A3grader.py


Testing actionsF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8])
('\n--- 5/5 points. Your actionsF_8p correctly returned', [('left', 1), ('right', 1), ('up', 1)])

Testing takeActionF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], (up, 1))
('\n--- 5/5 points. Your takeActionsF_8p correctly returned', ([1, 2, 3, 4, 0, 6, 7, 5, 8], 1))

Testing goalTestF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 5, 6, 7, 0, 8])

--- 5/5 points. Your goalTestF_8p correctly True

Testing aStarSearch([1, 2, 3, 4, 5, 6, 7, 0, 8],
                     actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]),
                     lambda s: h1_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]))
('\n--- 20/20 points. Your search correctly returned', ([[1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 0, 6, 7, 5, 8], [1, 2, 3, 0, 4, 6, 7, 5, 8], [0, 2, 3, 1, 4, 6, 7, 5, 8]], 3))

Testing iterativeDeepeningSearch([5, 2, 8, 0, 1, 4, 3, 7, 6], 
                                 [0, 2, 3, 1, 4,  6, 7, 5, 8],
          